#Downloading Dataset

In [0]:
! pip install -q kaggle

### for google colab

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d dataturks/resume-entities-for-ner

In [0]:
!unzip resume-entities-for-ner.zip

In [0]:
!pip install tensorflow-gpu==1.13.1

#imports

In [0]:
%matplotlib inline
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import re
import gc
import os
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report

In [0]:
import logging
tf.get_logger().setLevel(logging.ERROR)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

#Preprocessing Data


In [0]:
from collections import defaultdict
base_json = 'Entity Recognition in Resumes.json'
def pop_annot(raw_line):
    in_line = defaultdict(list, **raw_line)
    if 'annotation' in in_line:
        labels = in_line['annotation']
        for c_lab in labels:
            if len(c_lab['label'])>0:
                in_line[c_lab['label'][0]] += c_lab['points']
    return in_line
with open(base_json, 'r') as f:
    resume_data = [json.loads(f_line) for f_line in f.readlines()]
    resume_df = pd.DataFrame([pop_annot(line) for line in resume_data])
resume_df['length'] = resume_df['content'].map(len)

In [14]:
resume_df.head()

,content,annotation,extras,Skills,College Name,Graduation Year,Designation,Companies worked at,Email Address,Location,Name,Degree,Years of Experience,UNKNOWN,length
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",None,"[{'start': 1295, 'end': 1621, 'text': ' • Prog...","[{'start': 939, 'end': 956, 'text': 'Kendriya ...","[{'start': 856, 'end': 860, 'text': '2017 '}]","[{'start': 727, 'end': 769, 'text': 'B.E in In...","[{'start': 407, 'end': 415, 'text': 'Accenture...","[{'start': 95, 'end': 145, 'text': 'Indeed: in...","[{'start': 60, 'end': 68, 'text': 'Bengaluru'}]","[{'start': 0, 'end': 11, 'text': 'Abhishek Jha'}]",NaN,NaN,NaN,1622
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",None,"[{'start': 743, 'end': 1140, 'text': 'Database...","[{'start': 675, 'end': 702, 'text': 'Shivaji U...","[{'start': 729, 'end': 732, 'text': '2016'}, {...",NaN,NaN,"[{'start': 1155, 'end': 1198, 'text': 'indeed....","[{'start': 62, 'end': 67, 'text': 'Sangli'}]","[{'start': 0, 'end': 13, 'text': 'Afreen Jamad...","[{'start': 631, 'end': 672, 'text': 'Bachelor ...",NaN,NaN,1240
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",None,"[{'start': 3749, 'end': 3756, 'text': 'Teradat...","[{'start': 3421, 'end': 3457, 'text': 'Anurag ...",NaN,"[{'start': 1620, 'end': 1642, 'text': 'Senior ...","[{'start': 1645, 'end': 1659, 'text': 'Infosys...","[{'start': 65, 'end': 116, 'text': 'indeed.com...","[{'start': 1664, 'end': 1672, 'text': 'Hyderab...","[{'start': 0, 'end': 20, 'text': 'Akhil Yadav ...","[{'start': 3381, 'end': 3419, 'text': 'Electri...",NaN,NaN,4264
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",None,"[{'start': 8098, 'end': 8383, 'text': '❖ Opera...","[{'start': 7955, 'end': 7991, 'text': 'Indira ...","[{'start': 7994, 'end': 7997, 'text': '2012'}]","[{'start': 1472, 'end': 1509, 'text': 'Operati...","[{'start': 1512, 'end': 1517, 'text': 'UNISYS'...","[{'start': 2391, 'end': 2432, 'text': 'indeed....","[{'start': 5016, 'end': 5024, 'text': 'Bengalu...","[{'start': 0, 'end': 11, 'text': 'Alok Khandai'}]",NaN,NaN,NaN,8384
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",None,"[{'start': 973, 'end': 1702, 'text': 'EARCH EN...","[{'start': 922, 'end': 938, 'text': 'Mumbai Un...",NaN,"[{'start': 795, 'end': 802, 'text': 'lecturer'...","[{'start': 24, 'end': 40, 'text': ' oracle tut...","[{'start': 85, 'end': 129, 'text': 'indeed.com...","[{'start': 943, 'end': 948, 'text': 'Mumbai'},...","[{'start': 0, 'end': 12, 'text': 'Ananya Chava...","[{'start': 2010, 'end': 2012, 'text': 'MCA'}, ...",NaN,NaN,2933


In [0]:
resume1 = resume_df.iloc[0].content

In [0]:
def extract_higlights(raw_line):
    in_line = defaultdict(list, **raw_line)
    if 'annotation' in in_line:
        labels = in_line['annotation']
        for c_lab in labels:
            if len(c_lab['label'])>0:
                in_line['highlight'] += [dict(category = c_lab['label'][0], **cpts) for cpts in c_lab['points']]
    return in_line
resume_hl_df = pd.DataFrame([extract_higlights(line) for line in resume_data])
resume_hl_df['length'] = resume_hl_df['content'].map(len)

In [17]:
df = resume_hl_df['highlight']
df.head()

0    [{'category': 'Skills', 'start': 1295, 'end': ...
1    [{'category': 'Email Address', 'start': 1155, ...
2    [{'category': 'Skills', 'start': 3749, 'end': ...
3    [{'category': 'Skills', 'start': 8098, 'end': ...
4    [{'category': 'Degree', 'start': 2010, 'end': ...
Name: highlight, dtype: object

In [0]:
features = {
    "Name" : [],
    "College Name" : [],
    "Degree" : [],
    "Graduation Year" : [],
    "Years of Experience" : [],
    "Companies worked at" : [],
    "Designation" : [],
    "Skills" : [],
    "Location" : [],
    "Email Address" : []
}

In [0]:
for lis in resume_hl_df['highlight']:
  for cur_dict in lis:
    if(cur_dict['category'] != 'UNKNOWN'):
      features[cur_dict['category']].append(cur_dict['text'])

In [0]:
proper_features = {
    "Name" : [],
    "College Name" : [],
    "Degree" : [],
    "Graduation Year" : [],
    "Years of Experience" : [],
    "Companies worked at" : [],
    "Designation" : [],
    "Skills" : [],
    "Location" : [],
    "Email Address" : []
}

In [0]:
for key, lis in features.items():
  proper_features[key] = [x.strip().rstrip().replace('\n', '') for x in lis]

In [0]:
# preproces Email Address
proper_features['Email Address'] = [x.replace('Indeed: ', '') for x in proper_features['Email Address']]

In [0]:
# to preprocess skills
proper_features['Skills'] = [''.join([i if ord(i) < 128 else ', ' for i in x]).split(',') for x in proper_features['Skills']]
proper_features['Skills'] = [string.strip() for lis in proper_features['Skills'] for string in lis]
proper_features['Skills'] = list(filter(None, proper_features['Skills'])) 

In [24]:
proper_features['Skills']

['Programming language: C',
 'C++',
 'Java',
 'Oracle PeopleSoft',
 'Internet Of Things',
 'Machine Learning',
 'Database Management System',
 'Computer Networks',
 'Operating System worked on: Linux',
 'Windows',
 'MacNon - Technical Skills',
 'Honest and Hard-Working',
 'Tolerant and Flexible to Different Situations',
 'Polite and Calm',
 'Team-Player',
 'C (Less than 1 year)',
 'Database (Less than 1 year)',
 'Database Management (Less than 1 year)',
 'Database Management System (Less than 1 year)',
 'Java (Less than 1 year)',
 'Database (Less than 1 year)',
 'HTML (Less than 1 year)',
 'Linux. (Less than 1 year)',
 'MICROSOFTACCESS (Less than 1 year)',
 'MICROSOFT WINDOWS (Less than 1 year)ADDITIONAL INFORMATIONTECHNICAL SKILLS:',
 'Programming Languages: C',
 'C++',
 'Java',
 '.net',
 'php.',
 'Web Designing: HTML',
 'XML',
 'Operating Systems: Windows [',
 '] Windows Server 2003',
 'Linux.',
 'Database: MS Access',
 'MS SQL Server 2008',
 'Oracle 10g',
 'MySql.',
 'Teradata',
 'M

In [0]:
sentences =[]
labels = []

In [0]:
for lab in proper_features:
  for sen in proper_features[lab]:
    sentences.append(sen)
    labels.append(lab)


In [27]:
len(labels)

5484

In [0]:
tags = {
    "Name": 0,
    "College Name": 1,
    "Degree": 2,
    "Graduation Year": 3,
    "Years of Experience": 4,
    "Companies worked at": 5,
    "Designation": 6,
    "Skills": 7,
    "Location": 8,
    "Email Address": 9,
}

decode = {v: k for k, v in tags.items()} 

In [0]:
x = [[0,0,0,0,0,0,0,0,0,0] for i in range(len(labels))]
for i in range(len(labels)):
  x[i][tags[labels[i]]] = 1

In [0]:
df = pd.DataFrame(sentences, columns =['text']) 
df['english_labels'] = labels
df['labels'] = x

In [31]:
x_train_text = [sentences[i] for i in range(len(labels))]
x_train_labels = [tags[labels[i]] for i in range(len(labels))]

for i in range(0, len(labels), 100):
  print(labels[i], x_train_text[i], x_train_labels[i])

Name Abhishek Jha 0
Name Vikas Singh 0
Name Pavithra M 0
College Name Lovely Professional University 1
College Name Nagarjuna University 1
College Name Nagpur Polytechnic -  Nagpur, Maharashtra 1
Degree CBSE 2
Degree B.Tech in Electrical and Electronics Engineering 2
Degree B.SC. 2
Graduation Year 2011 3
Graduation Year 2013 3
Graduation Year 2014 3
Companies worked at Microsoft 5
Companies worked at Accenture 5
Companies worked at Oracle Inc 5
Companies worked at SAP ABAP 5
Companies worked at Microsoft 5
Companies worked at Microsoft 5
Companies worked at Infosys Ltd 5
Designation Team Lead 6
Designation fresher job 6
Designation Consultant 6
Designation software 6
Designation Senior Process Executive 6
Skills Programming language: C 7
Skills Jenkins 7
Skills XML 7
Skills java servlet 7
Skills knowledge of Active Directory 7
Skills UiPathMethodologies STLC 7
Skills Oracle 7
Skills HTML 7
Skills CHEF 7
Skills jenkins 7
Skills Open office 7
Skills Control-M 7
Skills Ethernet (3 years) 

In [32]:
main_df = pd.DataFrame({'Text': x_train_text, 'Label': x_train_labels})
main_df.head()

,Text,Label
0,Abhishek Jha,0
1,Afreen Jamadar,0
2,Akhil Yadav Polemaina,0
3,Alok Khandai,0
4,Ananya Chavan,0


In [33]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2020-04-14 07:53:03--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 2a00:1450:400c:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  74.6MB/s    in 5.2s    

2020-04-14 07:53:08 (74.6 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

--2020-04-14 07:53:10--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text

In [0]:
import modeling
import optimization
import run_classifier
import tokenization

In [0]:
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [36]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: model_folder/outputs
>>  BERT pretrained directory: model_folder/uncased_L-12_H-768_A-12


In [37]:
df2 = main_df
df2.head()

,Text,Label
0,Abhishek Jha,0
1,Afreen Jamadar,0
2,Akhil Yadav Polemaina,0
3,Alok Khandai,0
4,Ananya Chavan,0


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df2["Text"].values, df2["Label"].values, test_size=0.2, random_state=42)

In [39]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 50
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(10)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [40]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Please wait...
>> Started training at 2020-04-14 07:53:29.680857 
  Num examples = 4387
  Batch size = 32
>> Finished training at 2020-04-14 07:59:43.093837


In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [43]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}


In [0]:
def get_operable(resume_list):
  resume_list = list(filter(None, resume1.split('\n'))) 
  resume_list = [''.join([i if ord(i) < 128 else ', ' for i in x]).split(',') for x in resume_list]
  resume_list = [string.strip() for lis in resume_list for string in lis]
  resume_list = list(filter(None, resume_list))
  resume_list = [x.replace('Indeed: ', '') for x in resume_list]


  predict_examples = create_examples(resume_list, 'predict')

  predict_features = run_classifier.convert_examples_to_features(
      predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  predict_input_fn = input_fn_builder(
      features=predict_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=False)

  result = estimator.predict(input_fn=predict_input_fn)

  probs = []
  answer = []
  query_list = []

  for (query, prediction) in zip(resume_list, result):
    probs.append(np.max(prediction['probabilities']))
    answer.append(decode[np.argmax(prediction['probabilities'])])
    query_list.append(query)
  
  import itertools 

  for (prob, query, ans) in zip(probs, query_list, answer):
    if(ans=='Skills' and prob < 0.99):
      continue
    print("[Label: {0: <20}]<->[Entity: {1}]\n".format(ans, query))


# Performance

In [0]:
from sklearn.metrics import accuracy_score

In [46]:
print("Accuracy of BERT is:",accuracy_score(y_test,preds))

Accuracy of BERT is: 0.9617137648131268


In [47]:
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        48
           1       0.96      0.84      0.89        79
           2       0.86      0.89      0.87        61
           3       1.00      1.00      1.00        51
           4       1.00      1.00      1.00         7
           5       0.94      0.96      0.95       154
           6       0.98      0.96      0.97        89
           7       0.97      0.97      0.97       470
           8       0.99      1.00      0.99        83
           9       1.00      1.00      1.00        55

    accuracy                           0.96      1097
   macro avg       0.97      0.96      0.96      1097
weighted avg       0.96      0.96      0.96      1097



#Working

In [0]:
import random 


#choose a random resume
resume_number = random.randint(0, 220)
resume1 = resume_df.iloc[resume_number].content

In [55]:
print(resume1)

Shiksha Bhatnagar
chnadigarh - Email me on Indeed: indeed.com/r/Shiksha-Bhatnagar/70e68b28225ca499

WORK EXPERIENCE

online job in home

Microsoft and copy past -  Chandigarh, Chandigarh -

August 2016 to July 2017

i need a online job so that i can attend 
my regular college and i want to earn money that's it a part time online job so that i can do it
on my phone or laptop

EDUCATION

pass 12 in medical

chandigarh university -  Chandigarh, Chandigarh

September 2016 to August 2019

SKILLS

Microsoft office and java (Less than 1 year)

ADDITIONAL INFORMATION

i want to earn money by my hard work or smart work p

https://www.indeed.com/r/Shiksha-Bhatnagar/70e68b28225ca499?isid=rex-download&ikw=download-top&co=IN


In [56]:
get_operable(resume1)

{}
[Label: Name                ]<->[Entity: Shiksha Bhatnagar]

[Label: Email Address       ]<->[Entity: chnadigarh - Email me on indeed.com/r/Shiksha-Bhatnagar/70e68b28225ca499]

[Label: Skills              ]<->[Entity: WORK EXPERIENCE]

[Label: Designation         ]<->[Entity: online job in home]

[Label: Companies worked at ]<->[Entity: Microsoft and copy past -  Chandigarh]

[Label: Location            ]<->[Entity: Chandigarh -]

[Label: Degree              ]<->[Entity: pass 12 in medical]

[Label: College Name        ]<->[Entity: chandigarh university -  Chandigarh]

[Label: Location            ]<->[Entity: Chandigarh]

[Label: Skills              ]<->[Entity: SKILLS]

[Label: Skills              ]<->[Entity: Microsoft office and java (Less than 1 year)]

